In [8]:
from Helpers import helpers

In [9]:
from sklearn.datasets import make_blobs

n_samples = 2000
X, y_true = make_blobs(n_samples=n_samples, centers=4, n_features=3,
                       cluster_std=0.60, random_state=0)
X.shape

(2000, 3)

In [10]:
_, perturbed_df = helpers.load_plain_and_perturbed_dataset(3, import_path='./export/', perturbed_path='./export/truncated/')

In [17]:

import numpy as np
from scipy.stats import gamma

def generate_unit_sphere(): 
    vector = np.random.randn(3)
    vector /= np.linalg.norm(vector)

    #polar_angle = np.arccos(vector[2])
    #azimuth = np.arctan2(vector[1], vector[0])
    theta = 2 * np.random.uniform(0, np.pi)
    psi = np.arccos(2*np.random.uniform() - 1)
    return theta, psi, vector

def generate_3D_noise_for_dataset(X, epsilon):
    Z = []
    for x in X:
        noise = generate_3D_noise(epsilon)
        z = x + noise
        Z.append(z)
    return Z
def generate_3D_noise(epsilon): 
    polar_angle, azimuth, _ = generate_unit_sphere() # theta, psi
    r = gamma.rvs(3, scale=1/epsilon)
    # theta = 2 * np.pi * u[0]
    #theta = np.random.rand() * np.pi
    #phi = np.arccos(2 * u[1] - 1)
    #phi = np.random.rand() * np.pi*2 # 
    # https://mathworld.wolfram.com/SphericalCoordinates.html formula 4/5/6
    x = r * np.sin(polar_angle) * np.sin(azimuth)
    y = r * np.sin(polar_angle) * np.cos(azimuth)
    z = r * np.cos(polar_angle)
    return x, y, z

def remap_to_closted(perturbed_dataset, original_dataset, grid): 
    X, Y, Z = grid

        # Define the domain of the original dataset
    X_min, X_max = original_dataset[:, 0].min(), original_dataset[:, 0].max()
    Y_min, Y_max = original_dataset[:, 1].min(), original_dataset[:, 1].max()
    Z_min, Z_max = original_dataset[:, 2].min(), original_dataset[:, 2].max()
    domain_X = ((X_min, X_max), (Y_min, Y_max), (Z_min, Z_max))

    # Find the indices of the closest points in the original dataset for each point in the perturbed dataset
    indices_X = np.argmin(np.sum((perturbed_dataset[:, np.newaxis] - original_dataset[np.newaxis, :]) ** 2, axis=-1), axis=-1)

    # Check which points in the perturbed dataset are outside the domain of the original dataset
    outside_domain_X = np.logical_or(perturbed_dataset[:, 0] < domain_X[0][0], perturbed_dataset[:, 0] > domain_X[0][1])
    outside_domain_X = np.logical_or(outside_domain_X, perturbed_dataset[:, 1] < domain_X[1][0])
    outside_domain_X = np.logical_or(outside_domain_X, perturbed_dataset[:, 1] > domain_X[1][1])
    outside_domain_X = np.logical_or(outside_domain_X, perturbed_dataset[:, 2] < domain_X[2][0])
    outside_domain_X = np.logical_or(outside_domain_X, perturbed_dataset[:, 2] > domain_X[2][1])

    # Find the indices of the closest points in the meshgrid for each point in the perturbed dataset
    indices_M = np.argmin(np.sum((perturbed_dataset[:, np.newaxis] - np.array([X.ravel(), Y.ravel(), Z.ravel()]).T[np.newaxis, :]) ** 2, axis=-1), axis=-1)
    indices_M = np.unravel_index(indices_M, X.shape)

    # Check which points in the perturbed dataset are outside the domain of the meshgrid
    outside_domain_M = np.logical_or(perturbed_dataset[:, 0] < X_min, perturbed_dataset[:, 0] > X_max)
    outside_domain_M = np.logical_or(outside_domain_M, perturbed_dataset[:, 1] < Y_min)
    outside_domain_M = np.logical_or(outside_domain_M, perturbed_dataset[:, 1] > Y_max)
    outside_domain_M = np.logical_or(outside_domain_M, perturbed_dataset[:, 2] < Z_min)
    outside_domain_M = np.logical_or(outside_domain_M, perturbed_dataset[:, 2] > Z_max)

    # Remap the points outside the domain of the original dataset to the closest points in the original dataset
    remapped_dataset = perturbed_dataset.copy()
    remapped_dataset[outside_domain_X, :] = original_dataset[indices_X[outside_domain_X], :]

    # Remap the points outside the domain of the meshgrid to the closest points in the meshgrid
    remapped_dataset[outside_domain_M, 0] = X[indices_M][outside_domain_M]
    remapped_dataset[outside_domain_M, 1] = Y[indices_M][outside_domain_M]
    remapped_dataset[outside_domain_M, 2] = Z[indices_M][outside_domain_M]
    return remapped_dataset

def generate_truncated_perturbed_dataset(X, epsilon):
    meshgrid = np.meshgrid(np.linspace(X[:, 0].min(), X[:, 0].max(), num=6), np.linspace(X[:, 1].min(), X[:, 1].max(), num=6), np.linspace(X[:, 2].min(), X[:, 2].max(), num=6), indexing='ij')

    Z = generate_3D_noise_for_dataset(X, epsilon)
    Z = np.array(Z)
    Z = remap_to_closted(Z, X, meshgrid)
    return Z

In [12]:
test_df = helpers.run_mi_experiments(X, y_true, helpers.get_experiment_epsilons(), n_times=20, algorithm=generate_3D_noise_for_dataset)

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.064 0.012


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


1.0 1.0


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


...

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.592 0.396


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.512 0.356


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
test_df.to_csv('./export/privacy/report-advantages-' + str(n_samples) + '.csv')

In [19]:
truncated = helpers.run_mi_experiments(X, y_true, helpers.get_experiment_epsilons(), n_times=20, algorithm=generate_truncated_perturbed_dataset)

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.008 0.0


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.016 0.0


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


...

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.632 0.584


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


0.548 0.496


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
truncated.to_csv('./export/privacy/truncated-report-advantages-' + str(n_samples) + '.csv')